In [1]:
import pandas as pd
from sqlalchemy import create_engine
from secret import username, password

In [2]:
#import csv - Athletes

athlete_file = "../Resources/athlete_events.csv"
athlete_data_df = pd.read_csv(athlete_file)
athlete_data_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [3]:
#import csv - Country

population_file = "../Resources/population_GDP.csv"
population_data_df = pd.read_csv(population_file)
population_data = pd.DataFrame(population_data_df)
population_data.head()

,Country,Code,Population,GDP per Capita
0,Afghanistan,AFG,"32,526,562.00",594.323081
1,Albania,ALB,"2,889,167.00",3945.217582
2,Algeria,ALG,"39,666,519.00",4206.031232
3,American Samoa*,ASA,"55,538.00",NaN
4,Andorra,AND,"70,473.00",NaN


In [40]:
#create Athlete_event

athlete_event= athlete_data_df[['Medal']]

athlete_event

,Medal
0,NaN
1,NaN
2,NaN
3,Gold
4,NaN
...,...
271111,NaN
271112,NaN
271113,NaN
271114,NaN


In [5]:
#create country df

country_pd = population_data[['Code','Country']]

country_rename = country_pd.rename(columns={'Code':'country_code', 'Country':'country'})

country_rename['id'] = ""


country = country_rename[['country_code','country']]

country

,country_code,country
0,AFG,Afghanistan
1,ALB,Albania
2,ALG,Algeria
3,ASA,American Samoa*
4,AND,Andorra
...,...,...
196,VIE,Vietnam
197,ISV,Virgin Islands*
198,YEM,Yemen
199,ZAM,Zambia


In [6]:
#create country_stat

country_stat = population_data[[' Population ','GDP per Capita']]

country_stat_rename = country_stat.rename(columns={' Population ': 'population', 'GDP per Capita':'gdp'})

country_stat_rename.set_index('population')

,gdp
population,
"32,526,562.00",594.323081
"2,889,167.00",3945.217582
"39,666,519.00",4206.031232
"55,538.00",NaN
"70,473.00",NaN
...,...
"91,703,800.00",2111.138024
"103,574.00",NaN
"26,832,215.00",1406.291651


In [7]:
country_stat_rename["year"] = "2014"
country_stat_rename["country_id"] = ""

In [23]:
country_stats = country_stat_rename[["year","population", "gdp"]] 

country_stats.set_index('year')


country_stats['population'] = country_stats['population'].str.strip()

country_stats


<ipython-input-23-c236039fb71e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_stats['population'] = country_stats['population'].str.strip()


,year,population,gdp
0,2014,"32,526,562.00",594.323081
1,2014,"2,889,167.00",3945.217582
2,2014,"39,666,519.00",4206.031232
3,2014,"55,538.00",NaN
4,2014,"70,473.00",NaN
...,...,...,...
196,2014,"91,703,800.00",2111.138024
197,2014,"103,574.00",NaN
198,2014,"26,832,215.00",1406.291651
199,2014,"16,211,767.00",1304.879014


In [24]:
country_stats['population'] = country_stats['population'].str.replace(",","")

country_stats

<ipython-input-24-55b1776cf577>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_stats['population'] = country_stats['population'].str.replace(",","")


,year,population,gdp
0,2014,32526562.00,594.323081
1,2014,2889167.00,3945.217582
2,2014,39666519.00,4206.031232
3,2014,55538.00,NaN
4,2014,70473.00,NaN
...,...,...,...
196,2014,91703800.00,2111.138024
197,2014,103574.00,NaN
198,2014,26832215.00,1406.291651
199,2014,16211767.00,1304.879014


In [25]:
country_stats['population'] = country_stats['population'].str.replace(".00","")

country_stats_update = country_stats.fillna(0)


#use this one
country_stats_update

<ipython-input-25-9f5816efbd02>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_stats['population'] = country_stats['population'].str.replace(".00","")


,year,population,gdp
0,2014,32526562,594.323081
1,2014,2889167,3945.217582
2,2014,39666519,4206.031232
3,2014,55538,0.000000
4,2014,70473,0.000000
...,...,...,...
196,2014,91703,2111.138024
197,2014,103574,0.000000
198,2014,26832215,1406.291651
199,2014,16211767,1304.879014


In [ ]:
country_stats_update.astype(int)

In [ ]:
country_stats_update.info()

In [34]:
#connect to db
rds_connection_string = f'{username}:{password}@localhost:5432/Olympics_db'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [35]:
engine.table_names()

['country',
 'athlete',
 'event',
 'sport',
 'olympic_season',
 'athlete_event',
 'country_stats']

In [36]:
country.to_sql(name='country', con=engine, if_exists='append', index=False)

In [37]:
pd.read_sql_query('select * from country', con=engine).head()

,id,country_code,country
0,1,AFG,Afghanistan
1,2,ALB,Albania
2,3,ALG,Algeria
3,4,ASA,American Samoa*
4,5,AND,Andorra


In [38]:
country_stats_update.to_sql(name='country_stats', con=engine, if_exists='append', index=False)

In [39]:
pd.read_sql_query('select * from country_stats', con=engine).head()

,year,country_id,population,gdp
0,2014,None,32526562,594.323081
1,2014,None,2889167,3945.217582
2,2014,None,39666519,4206.031232
3,2014,None,55538,0.000000
4,2014,None,70473,0.000000
